In [ ]:
!pip install ollama


In [ ]:
import ollama
response = ollama.chat(
    model="deepseek-r1:1.5b",
    messages=[
        {"role": "user", "content": "Explain Newton's second law of motion"},
    ],
)
print(response["message"]["content"])

In [1]:
import os
from typing import List, Dict
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA

: 

In [ ]:

class PDFRAGSystem:
    def __init__(self, model_name: str = "llama2"):
        """
        Initialize the RAG system with specified Ollama model.
        
        Args:
            model_name (str): Name of the Ollama model to use
        """
        self.model_name = model_name
        self.embeddings = OllamaEmbeddings(model=model_name)
        self.llm = Ollama(model=model_name)
        self.vector_store = None
        
    def load_pdfs(self, pdf_directory: str) -> List[str]:
        """
        Load all PDFs from the specified directory.
        
        Args:
            pdf_directory (str): Path to directory containing PDF files
            
        Returns:
            List[str]: List of extracted text from PDFs
        """
        documents = []
        for filename in os.listdir(pdf_directory):
            if filename.endswith('.pdf'):
                file_path = os.path.join(pdf_directory, filename)
                loader = PyPDFLoader(file_path)
                documents.extend(loader.load())
                
        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
        
        return text_splitter.split_documents(documents)
    
    def create_vector_store(self, documents: List[str]):
        """
        Create FAISS vector store from documents.
        
        Args:
            documents (List[str]): List of document chunks
        """
        self.vector_store = FAISS.from_documents(
            documents,
            self.embeddings
        )
    
    def setup_qa_chain(self) -> RetrievalQA:
        """
        Set up the question-answering chain.
        
        Returns:
            RetrievalQA: The QA chain object
        """
        return RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vector_store.as_retriever(),
            return_source_documents=True
        )
    
    def query(self, question: str) -> Dict:
        """
        Query the RAG system.
        
        Args:
            question (str): Question to ask about the PDFs
            
        Returns:
            Dict: Response containing answer and source documents
        """
        if not self.vector_store:
            raise ValueError("Vector store not initialized. Please load documents first.")
            
        qa_chain = self.setup_qa_chain()
        return qa_chain({"query": question})

def main():
    # Initialize the RAG system
    rag_system = PDFRAGSystem(model_name="llama2")
    
    # Load PDFs from directory
    pdf_dir = "path/to/your/pdfs"
    documents = rag_system.load_pdfs(pdf_dir)
    
    # Create vector store
    rag_system.create_vector_store(documents)
    
    # Example query
    question = "What are the main topics discussed in these documents?"
    response = rag_system.query(question)
    
    print("Answer:", response["result"])
    print("\nSources:")
    for doc in response["source_documents"]:
        print(f"- {doc.metadata['source']}, page {doc.metadata['page']}")

if __name__ == "__main__":
    main()